# Install Packages

In [9]:
pip install fastapi uvicorn openai chromadb pypdf2


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\DELL\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Read Data from pdf files

In [1]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(file_path):
    global text_list,page_ids
    text_list = []

    pdf_reader = PdfReader(file_path)

    for page in pdf_reader.pages:
        text_list.append(page.extract_text())

    page_ids = list(range(1, len(text_list) + 1))
    page_ids = [str(i) for i in page_ids]

    return text_list




# ChromaDB integration 

In [2]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions


def add_to_chromadb(path):
    global index_pdf_1
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(api_key="sk-proj-FkcYFkpik9wmpDozUeKgT3BlbkFJ8iAdWO3AU4A6Mvt3CnNs",model_name = "text-embedding-ada-002")

    client = chromadb.PersistentClient(path=path)

    index_pdf_1 = client.get_or_create_collection(name="index_pdf_1",embedding_function=openai_ef)


    index_pdf_1.add(
        documents=text_list,
        ids=page_ids
    )

In [3]:
def get_data_from_vector_database(user_input):

    user_input= user_input

    result = index_pdf_1.query(
        query_texts=[user_input],
        n_results=3
    )
    context=result["documents"]
    context=context[0]
    return context

# Inputs

In [4]:

file_path = r"C:\Users\DELL\Documents\Batch-2\LAB-2\data_pdfs\Energy Communities.pdf"
text_list = extract_text_from_pdf(file_path)

local_path="vectordata_pdf"
add_to_chromadb(local_path)


# Create Fast Api

In [5]:

from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware 
from fastapi.responses import JSONResponse
from pathlib import Path
import sys
import os
import openai

sys.path.insert(0, 'path_to_project')


app = FastAPI()

# Setup CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allows all origins
    allow_credentials=True,
    allow_methods=["*"],  # Allows all methods
    allow_headers=["*"],  # Allows all headers
)

In [6]:
api_key = 'sk-proj-FkcYFkpik9wmpDozUeKgT3BlbkFJ8iAdWO3AU4A6Mvt3CnNs'

def ask_openai(question, api_key):
    context=get_data_from_vector_database(question)
    
    openai.api_key = api_key
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Use the updated model name
        messages=[
            {"role": "system", "content": f"You are a very helpful chat assistant who answers the user_input with the context provide. Only answer based on the provided context dont add any other details"},
            {"role": "user", f"content":f"user_input: {question},context:{context}" },
        ],
        max_tokens=500,  # Adjust the token limit as needed
        n=1,
        stop=None,
        temperature=0.7
    )

    answer = response['choices'][0]['message']['content'].strip()
    return answer

In [7]:

@app.get("/ask")
async def process_images(question: str):
    """
    Endpoint to process images.
    """
    try:
        result = ask_openai(question, api_key)
        return {"question": question, "output": result}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


# Run fast Api

In [8]:
import uvicorn
import threading
import time

def run():
    uvicorn.run(app, host="0.0.0.0", port=8001)

# Event to keep the server running
stop_event = threading.Event()

# Start the server in a separate thread
thread = threading.Thread(target=run)
thread.start()

# Cell 6: Keep the main thread alive until stop_event is set
try:
    while not stop_event.is_set():
        time.sleep(1)
except KeyboardInterrupt:
    stop_event.set()
    thread.join()

INFO:     Started server process [45712]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


INFO:     127.0.0.1:59977 - "GET /ask?question=ENERGY%20COMMUNITY%20CATEGORIES HTTP/1.1" 200 OK
INFO:     127.0.0.1:59990 - "GET /ask?question=ENERGY%20COMMUNITY%20CATEGORIES HTTP/1.1" 200 OK


In [ ]:
# http://127.0.0.1:8001/ask?question=ENERGY COMMUNITY CATEGORIES